## Naming conventions :

In [1]:
# as we write python code then:
# 1- variables : lowercase seperated by underscore (my_variable)
# 2- methods : lowercase seperated by underscore (my_function)
# 3- constants : ALLCAPITAL

## Packages :

In [ ]:
#import OS to share the project dir
import os
os.chdir('') # put here your dire that contains this notebook

#import important packages
#import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import svm
import cv2 as cv

#packages for visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#allows charts to appear in the notebook 
%matplotlib inline

#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

## Reading Data :

In [ ]:
# we use IAM handwriting database download it first
# here we will read the images and get the important info about the images 
# we divide the available images into (train,cv and test)
# division will be depend on the number of the images 

images=0  # contain the whole images 
train_images=0 # only the train 
cv_images=0  # cross validation 
test_images=0 # test images 
WIDTH=0 #image width
HEIGHT=0 #image height

## Preprocessing Module:
#### (database has different shapes assume we will use words shape)

### 1-Gray scale convertor:

In [7]:
#this function convert the image to a gray scale image 
def gray_scale (image):
    
    return image

### 2-Binarization:

In [8]:
# this function convert from gray scale image to binary image 
# in this function we assume that the word will be black and the background will be wight
def binarization(image):
    
    return image

### 3-Morphological Filter: 

In [9]:
# this function will remove the noise from the images 
# one of the best filters to remove the noise is the "opening filter"
# "opening filter" is a max filter followed by minimum filter 
# note: if you have another filter that gives better accuracy ,use it for sure :D 

def max_filter (image):
    
    return image

In [10]:
def min_filter (image):
    
    return image

In [ ]:
def opening_filter(image):
    image=max_filter(image)
    image=min_filter(image)
    return image;

### 4-Crop the images:

In [12]:
#get the window that contains the words 
def crop_image(image):
    
    return image

### 5-Resize the images:

In [13]:
#note : use WIDTH and HEIGHT 
def resize_image (image):
    
    return image

### 6-Data augmentation:

In [15]:
#here we will implement augmentation function to have a large dataset 
#augmentation also good with lage dataset to increase the accuracy 
#so we will implement this function to check its improvement on the accuracy 
#if it do nothing then we will not use it 

#note: data augmentation means producing new images from a given one
      #by doing rotation and anything like so
def augmentation(image):
    
    return images

In [16]:
#note: this is my thoughts about the preprocessing phase if anyone has another idea to be added do it

# ----------------------------------------------------------------------------------------

## Feature Engineering:

In [17]:
#here we need to read the avaiable papers 
#to choose the techniques that will be used to get our feature  

In [ ]:
def sift (image_path):
    img = cv.imread(img_path)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.xfeatures2d.SIFT_create()
    kp = sift.detect(gray,None)
    img=cv.drawKeypoints(gray,kp,img)
    cv.imwrite('sift_keypoints.jpg',img)
    return 

## Training Module:

### 1-Bayes Classifer:

In [22]:
def bayes_classifer():
    
    return 

### 2-SVM :

In [23]:
def svm():
    
    return 

### 3-Linear/Logistic Regression:

In [24]:
def regression():
    return

### 4- K-NN:

In [25]:
def knn():
    return

### 5- Neural Network:

In [26]:
def neural_network():
    return

## Performance Module:

In [27]:
def get_accuracy():
    
    return

# Main Logic:

In [28]:
#here we will call the previous methods in a certain sequence 

